In [1]:
# 18-19.07.24
# vera.provatorova@dh.huc.knaw.nl

# Questions: which ships transported opium？How many times/voyages did these ships transport opium? What are the types of these ships?


In [ ]:
# Step 1: loading and exploring the data

In [2]:
import pandas as pd

In [16]:
das_file = pd.ExcelFile('data/das.xlsx') # downloaded at https://github.com/globalise-huygens/enrich_from_das/blob/master/das.xlsx

das_file.sheet_names

['das_voyage',
 'ship',
 'shipNameVariant',
 'shipType',
 'master',
 'onboard',
 'yard',
 'chamber',
 'place']

In [17]:
das_data = {name: pd.read_excel(das_file, name)
          for name in das_file.sheet_names}

In [30]:
# No info on commodities in DAS, so we need to connect it to BKB
# First step: read the BKB data
import glob

def get_name(path):
    '''
    in: bgb_cargo.tsv
    out: cargo
    '''
    return path.split('.')[0].split('_')[-1]

bkb_data = {get_name(path): pd.read_csv(path,sep='\t')
           for path in glob.glob('data/bkb_cargo_logs/*')}

/var/folders/zp/bd6xhyd10t3_hf1gm3jsc4m80000gn/T/ipykernel_18227/3831798763.py:12: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  bkb_data = {get_name(path): pd.read_csv(path,sep='\t')


In [ ]:
# Step 2: connecting the data

In [46]:
def get_voyages_for_commodity(commodity_name='opium'): # Looks for the DAS voyages
    # 1. Finding the ID of our commodity by its name
    df_commodity = bkb_data['product'][bkb_data['product']['naam']==commodity_name]
    if len(df_commodity) == 0:
        print('No ID matches found for ',commodity_name)
    commodity_id = df_commodity.iloc[0]['id']
    
    # 2. Finding the cargo data in BKB voyages using the commodity ID
    df_cargo = bkb_data['cargo'][bkb_data['cargo']['carProductId']==commodity_id]
    if len(df_cargo) == 0:
        print('No cargo data found for ',commodity_name)
        
    # 3. Finding the DAS data using the voyages info
    df_with_das_ids = df_cargo.merge(bkb_data['voyage'],
                                 left_on='carVoyageId', right_on='voyId').dropna(subset=['voyageDAS'])
    if len(df_with_das_ids) == 0:
        print('No DAS data found for ',commodity_name)
        
    return df_with_das_ids.reset_index()

In [78]:
def get_das_info(df_bkb): # extract the info needed to answer our questions
    # 1. Find the voyages info
    df_voyages = df_bkb.merge(das_data['das_voyage'],left_on='voyageDAS',right_on='voyId')
        
    # 2. Find ship names and types
    df_with_ship_names = df_voyages.merge(das_data['shipNameVariant'],
                                         on='shipID')
        
    df_with_ship_type_ids = df_with_ship_names.merge(das_data['ship'],on='shipID')
    
    df_with_ship_types = df_with_ship_type_ids.merge(das_data['shipType'],
                                                     left_on='voyTypeOfShipID',
                                                     right_on='shipTypeID')
    if len(df_with_ship_types) == 0:
        print('No data found on ship types')
        return df_with_ship_names
    
    return df_with_ship_types

In [94]:
bkb_data['cargo'][bkb_data['cargo']['carProductId']==1142].nunique()

carId                     234
carVoyageId               234
carProductId                1
carSpecificationId         25
carUnit                     4
carQuantity               154
carQuantityNumeric        153
carValue                   38
carValueGuldens            37
carValueStuivers           18
carValuePenningen           2
carValueLicht             181
carValueLichtGuldens      175
carValueLichtStuivers      20
carValueLichtPenningen      2
carRemarks                  4
carOrder                   44
changed_when              232
changed_by                 12
timestamp                   7
all_fields                 31
dtype: int64

In [91]:
df_opium = get_voyages_for_commodity('opium')
df_opium

,index,carId,carVoyageId,carProductId,carSpecificationId,carUnit,carQuantity,carQuantityNumeric,carValue,carValueGuldens,...,timestamp_y,voySourceId,voynumber,voyImage,voyRemarksForEndUser,voyDepartureRegioId,voyArrivalRegioId,voyFolioNummer,all_fields_y,first_ship_name
0,0,646738,99404,1142.0,781.0,69.0,4.359,4359,"21.075,7,8",21075.0,...,2013-09-10 14:30:24,148.0,50,NaN,NaN,3185,3129,3,50 Batavia Batavia Amsterdam Republiek 1780 ...,Overduin
1,1,646807,99410,1142.0,781.0,69.0,1.920,1920,"9.279,50",9279.0,...,2013-09-10 14:30:24,148.0,56,NaN,NaN,3185,3129,3,56 Batavia Batavia Amsterdam Republiek 1780 ...,Willem Frederik
2,6,663416,101181,1142.0,781.0,69.0,1.319,1319,"6.117,19",6117.0,...,2013-09-10 14:30:31,141.0,1801,NaN,NaN,3185,3129,5,1801 Batavia Batavia Amsterdam Republiek 1778...,Triton
3,13,663355,101170,1142.0,781.0,69.0,439,439,"2.034,13,8",2034.0,...,2013-09-10 14:30:31,141.0,1790,NaN,NaN,3185,3129,4,1790 Batavia Batavia Amsterdam Republiek 1778...,Dolfijn


In [89]:
df_ships = get_das_info(df_opium)
df_ships

No data found on ship types


,index,carId,carVoyageId,carProductId,carSpecificationId,carUnit,carQuantity,carQuantityNumeric,carValue,carValueGuldens,...,voyArrivalPlaceID,voyInvoiceValue_y,voyChamber2ID,voyParticulars,voyCorrespondingNumber,voyRGPDeel,voymaster_VOCOPVid,shipNameVariantID,shipNameVariant,shipNameVariantRemark
0,0,646738,99404,1142.0,781.0,69.0,4.359,4359,"21.075,7,8",21075.0,...,DAS_pl007,NaN,NaN,The following data refer to the crew of the OV...,8127,166,NaN,DAS_snv1169,Overduin,NaN
1,1,646807,99410,1142.0,781.0,69.0,1.920,1920,"9.279,50",9279.0,...,DAS_pl007,NaN,NaN,See the OVERDUIN (4344). The WILLEM FREDERIK w...,NaN,166,NaN,DAS_snv1746,Willem Frederik,NaN
2,6,663416,101181,1142.0,781.0,69.0,1.319,1319,"6.117,19",6117.0,...,DAS_pl007,NaN,NaN,See the MORGENSTER (4291).,8061,166,NaN,DAS_snv1500,Triton,NaN
3,13,663355,101170,1142.0,781.0,69.0,439,439,"2.034,13,8",2034.0,...,DAS_pl007,NaN,NaN,After having called at Batavia the DOLFIJN sai...,8030,166,NaN,DAS_snv0321,Dolfijn,NaN


In [90]:
# Showing the ship names and voyage info
df_ships[['voyageDAS','shipNameVariant','heenreis','terugreis']]

,voyageDAS,shipNameVariant,heenreis,terugreis
0,95395.0,Overduin,1,0
1,95397.0,Willem Frederik,1,0
2,95362.0,Triton,1,0
3,95349.0,Dolfijn,1,0
